# Access locations and times of Veery encounters

For this challenge, you will use a database called the [Global
Biodiversity Information Facility (GBIF)](https://www.gbif.org/). GBIF
is compiled from species observation data all over the world, and
includes everything from museum specimens to photos taken by citizen
scientists in their backyards.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Explore GBIF</div></div><div class="callout-body-container callout-body"><p>Before your get started, go to the <a
href="https://www.gbif.org/occurrence/search">GBIF occurrences search
page</a> and explore the data.</p></div></div>

> **Contribute to open data**
>
> You can get your own observations added to GBIF using
> [iNaturalist](https://www.inaturalist.org/)!

### Set up your code to prepare for download

We will be getting data from a source called [GBIF (Global Biodiversity
Information Facility)](https://www.gbif.org/). We need a package called
`pygbif` to access the data, which may not be included in your
environment. Install it by running the cell below:

In [ ]:
%%bash
pip install pygbif

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>In the imports cell, we’ve included some packages that you will need.
Add imports for packages that will help you:</p>
<ol type="1">
<li>Work with reproducible file paths</li>
<li>Work with tabular data</li>
</ol></div></div>

In [2]:
import time
import zipfile
from getpass import getpass
from glob import glob

import pygbif.occurrences as occ
import pygbif.species as species

import os # operating system and directory for working with files
import pathlib # best package for dealing with directories and files. Treat files paths as object
import geopandas as gpd
import pandas as pd

In [3]:
# Create data directory in the home folder
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'species_distribution',
)
os.makedirs(data_dir, exist_ok=True)

# Define the directory name for GBIF data
gbif_dir = os.path.join(data_dir, 'species_distribution')

:::

### Register and log in to GBIF

You will need a [GBIF account](https://www.gbif.org/) to complete this
challenge. You can use your GitHub account to authenticate with GBIF.
Then, run the following code to save your credentials on your computer.

> **Warning**
>
> Your email address **must** match the email you used to sign up for
> GBIF!

> **Tip**
>
> If you accidentally enter your credentials wrong, you can set
> `reset_credentials=True` instead of `reset_credentials=False`.

In [9]:
reset_credentials = False
# GBIF needs a username, password, and email
credentials = dict(
    GBIF_USER=(input, 'sear2701'),
    GBIF_PWD=(getpass, '2024Onwardup!'),
    GBIF_EMAIL=(input, 'wwa.arens@gmail.com'),
)
for env_variable, (prompt_func, prompt_text) in credentials.items():
    # Delete credential from environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    # Ask for credential and save to environment
    
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

### Get the species key

> ** Your task**
>
> 1.  Replace the `species_name` with the name of the species you want
>     to look up
> 2.  Run the code to get the species key

In [10]:
# Query species
species_info = species.name_lookup('Grus canadensis', rank='species')

# Get the first result
first_result = species_info['results'][0]

# Get the species key (nubKey)
species_key = first_result['nubKey']

# Check the result
first_result['species'], species_key

('Antigone canadensis', 2474953)

### Download data from GBIF

::: {.callout-task title=“Submit a request to GBIF”

1.  Replace `csv_file_pattern` with a string that will match **any**
    `.csv` file when used in the `glob` function. HINT: the character
    `*` represents any number of any values except the file separator
    (e.g. `/`)

2.  Add parameters to the GBIF download function, `occ.download()` to
    limit your query to:

    -   observations
    -   from 2023
    -   with spatial coordinates.

3.  Then, run the download. **This can take a few minutes**. :::

In [11]:
# Only download once
gbif_pattern = os.path.join(gbif_dir, '*.csv')
if not glob(gbif_pattern):
    # Submit query to GBIF
    gbif_query = occ.download([
        "speciesKey = 2474953",
        "year = 2023",
        "hasCoordinate = TRUE",
    ],
    user=credentials['GBIF_USER'][1], # not sure what lines10-12 do
    pwd=credentials['GBIF_PWD'][1], 
    email=credentials['GBIF_EMAIL'][1])

# Only download once
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

        # Wait for the download to build
        wait = occ.download_meta(os.environ['GBIF_DOWNLOAD_KEY'])['status'] 
        while not wait=='SUCCEEDED':
            wait = occ.download_meta(os.environ['GBIF_DOWNLOAD_KEY'])['status'] 
            time.sleep(5)

        # Download GBIF data
        download_info = occ.download_get(
            os.environ['GBIF_DOWNLOAD_KEY'], 
            path=data_dir)

        # Unzip GBIF data
        with zipfile.ZipFile(download_info['path']) as download_zip:
            download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path
gbif_path = glob(gbif_pattern)[0]

INFO:Your download key is 0005045-241007104925546
INFO:Download file size: 33101248 bytes
INFO:On disk at /home/jovyan/earth-analytics/data/species_distribution/0005045-241007104925546.zip


### Load the GBIF data into Python

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Load GBIF data</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Look at the beginning of the file you downloaded using the code
below. What do you think the <strong>delimiter</strong> is?</li>
<li>Run the following code cell. What happens?</li>
<li>Uncomment and modify the parameters of <code>pd.read_csv()</code>
below until your data loads successfully and you have only the columns
you want.</li>
</ol></div></div>

You can use the following code to look at the beginning of your file:

In [12]:
!head -n 2 $gbif_path 

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
4054868020	50c9509d-22c7-4a22-a47d-8c48425ef4a7	https://www.inaturalist.org/observations/150310051	Animalia	Chordata	Aves	Gruiformes	Gruidae	Grus	Grus canadensis		SPECIES	Antigone canadensis (Linnaeus, 1758)	Antigone canadensis		US		Texas	PRESENT		28eb1a3f-1c15-4a95-931a-4af90ecb574d	33.527007	-102.025136	250.0						2023-01-26T15:39	26	1	2023	9036008	2474953	HUM

In [19]:
# Load the GBIF data
gbif_df = pd.read_csv(
    gbif_path, 
    delimiter='\t',
    index_col='gbifID',
    usecols=['gbifID','scientificName','countryCode','stateProvince','decimalLatitude','decimalLongitude', 'month','year']
)
gbif_df.head(5)

,scientificName,countryCode,stateProvince,decimalLatitude,decimalLongitude,month,year
gbifID,,,,,,,
4054868020,"Antigone canadensis (Linnaeus, 1758)",US,Texas,33.527007,-102.025136,1,2023
4134153626,"Antigone canadensis (Linnaeus, 1758)",US,Wisconsin,44.937105,-89.656949,6,2023
4137871437,"Antigone canadensis (Linnaeus, 1758)",US,California,38.156053,-121.459555,3,2023
4046599887,"Antigone canadensis (Linnaeus, 1758)",US,Florida,28.517079,-81.449704,2,2023
4138596486,"Antigone canadensis (Linnaeus, 1758)",CA,Ontario,44.486880,-79.996687,6,2023


In [21]:
gbif_df.value_counts()

scientificName                                 countryCode  stateProvince  decimalLatitude  decimalLongitude  month  year
Antigone canadensis (Linnaeus, 1758)           US           Ohio           41.627710        -83.191890        5      2023    1565
                                                                           41.645070        -83.263720        5      2023     499
                                               CA           Ontario        41.955400        -82.514000        5      2023     390
                                               US           Wisconsin      43.033360        -89.351380        4      2023     340
                                                            Arizona        31.561499        -109.720020       1      2023     318
                                                                                                                             ... 
Antigone canadensis tabida (J.L.Peters, 1925)  US           Oregon         45.243952        -117.9

In [1]:
from nbconvert import HTMLExporter
import nbformat
# Define the path to your notebook
sandhill_data = '05-species-dist-sandhill-crane.ipynb'
# Read the notebook
with open(sandhill_data) as f:
    notebook_content = nbformat.read(f, as_version=4)
# Initialize HTMLExporter
html_exporter = HTMLExporter()
# Convert the notebook to HTML
html_content, resources = html_exporter.from_notebook_node(notebook_content)
# Save the HTML content to a file
with open('sandhill_data.html', 'w') as f:
    f.write(html_content)
print("Notebook exported as HTML successfully.")

Notebook exported as HTML successfully.
